In [39]:
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
import hopsworks
import hopsworks
import numpy as np
from hsml.schema import Schema
from hsml.model_schema import ModelSchema
import joblib
import os

In [4]:
project = hopsworks.login()
fs = project.get_feature_store()

Copy your Api Key (first register/login): https://c.app.hopsworks.ai/account/api/generated

Paste it here: ··········
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/371861
Connected. Call `.close()` to terminate connection gracefully.


In [11]:
earthquakes_fg = fs.get_feature_group(name="earthquakes", version=1)
query = earthquakes_fg.select_all()
feature_view = fs.get_or_create_feature_view(name="earthquakes", version=1, labels=["mag"], query=query)

In [12]:
X_train, X_test, y_train, y_test = feature_view.train_test_split(0.2)

Finished: Reading data from Hopsworks, using ArrowFlight (7.82s) 


In [13]:
X_train.drop(columns=['id', 'time'], inplace=True)
X_test.drop(columns=['id', 'time'], inplace=True)

In [14]:
X_train

,latitude,longitude,depth,deptherror,rms,reviewed
0,37.574500,-118.846333,-0.990000,0.400000,0.02,1.0
1,38.771667,-122.710503,1.500000,0.430000,0.02,0.0
3,36.546333,-89.711833,6.630000,0.450000,0.14,1.0
6,9.372300,93.809600,10.000000,1.717000,0.61,1.0
7,37.566500,-118.926167,8.990000,1.920000,0.01,1.0
...,...,...,...,...,...,...
130661,38.827500,-122.800667,2.730000,0.240000,0.04,1.0
130663,31.681126,-104.295363,7.802588,0.909312,0.10,1.0
130664,38.820167,-122.822998,2.030000,0.990000,0.01,0.0
130665,46.248500,-119.436167,10.470000,0.650000,0.06,1.0


In [34]:
model = HistGradientBoostingRegressor()

params = {'learning_rate': [0.1, 0.01, 0.001], 'l2_regularization': [0, 1e-5, 1e-3]}
gs = GridSearchCV(model, params, scoring='neg_mean_squared_error')
gs.fit(X_train, y_train.values.ravel())
gs.cv_results_

{'mean_fit_time': array([0.99591236, 1.7536921 , 1.63373141, 0.95619125, 1.58674254,
        1.12807593, 1.03423758, 1.38056459, 1.1003242 ]),
 'std_fit_time': array([0.02963415, 1.22864508, 1.07064647, 0.01907797, 1.03455515,
        0.11239332, 0.22348919, 0.48407787, 0.02123388]),
 'mean_score_time': array([0.13027992, 0.51710529, 0.14954071, 0.13361607, 0.14807444,
        0.47800093, 0.14057641, 0.38652124, 0.14745731]),
 'std_score_time': array([0.00358392, 0.72401241, 0.01014151, 0.01196614, 0.00876174,
        0.65404046, 0.01103224, 0.47988784, 0.00216483]),
 'param_l2_regularization': masked_array(data=[0, 0, 0, 1e-05, 1e-05, 1e-05, 0.001, 0.001, 0.001],
              mask=[False, False, False, False, False, False, False, False,
                    False],
        fill_value='?',
             dtype=object),
 'param_learning_rate': masked_array(data=[0.1, 0.01, 0.001, 0.1, 0.01, 0.001, 0.1, 0.01, 0.001],
              mask=[False, False, False, False, False, False, False, Fals

In [35]:
gs.best_estimator_

HistGradientBoostingRegressor(l2_regularization=1e-05)

In [36]:
gs.predict(X_train)

array([0.50510038, 0.91765414, 2.17658516, ..., 0.64717497, 0.61193665,
       1.66129996])

In [26]:
y_train.values.ravel()

array([0.28, 0.53, 3.12, ..., 0.76, 0.19, 1.66])

In [38]:
score = -gs.score(X_test, y_test.values.ravel())

In [40]:
mr = project.get_model_registry()

model_dir="earthquakes_model"
if os.path.isdir(model_dir) == False:
    os.mkdir(model_dir)

joblib.dump(gs.best_estimator_, model_dir + "/earthquakes_model.pkl")

input_schema = Schema(X_train)
output_schema = Schema(y_train)
model_schema = ModelSchema(input_schema, output_schema)

earthquakes_model = mr.python.create_model(
    name="earthquakes_model",
    metrics={"mse" : score},
    model_schema=model_schema,
    description="Earthquake Magnitude Predictor"
)

earthquakes_model.save(model_dir)

Connected. Call `.close()` to terminate connection gracefully.


  0%|          | 0/6 [00:00<?, ?it/s]

Uploading: 0.000%|          | 0/350168 elapsed<00:00 remaining<?

Uploading: 0.000%|          | 0/600 elapsed<00:00 remaining<?

Model created, explore it at https://c.app.hopsworks.ai:443/p/371861/models/earthquakes_model/1


Model(name: 'earthquakes_model', version: 1)